In [ ]:
from  datetime import datetime,timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb
import numpy as np

In [ ]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }


In [ ]:
prices = pd.read_csv(r"../input/m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)
for col, col_dtype in PRICE_DTYPES.items():
    if col_dtype == "category":
        prices[col] = prices[col].cat.codes.astype("int16")
        prices[col] -= prices[col].min()

cal = pd.read_csv(r"../input/m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
cal["date"] = pd.to_datetime(cal["date"])
for col, col_dtype in CAL_DTYPES.items():
    if col_dtype == "category":
        cal[col] = cal[col].cat.codes.astype("int16")
        cal[col] -= cal[col].min()
start_day = 1500
tr_last = 1913
# start_day = max(1 if is_train  else tr_last-max_lags, first_day)
numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
dtype = {numcol:"float32" for numcol in numcols} 
dtype.update({col: "category" for col in catcols if col != "id"})
dt1 = pd.read_csv(r"../input/m5-forecasting-accuracy/sales_train_validation.csv", 
                 usecols = catcols + numcols, dtype = dtype)
# dt2 = pd.read_csv(r"../input/m5-forecasting-accuracy/sales_train_validation.csv")
for col in catcols:
    if col != "id":
        dt1[col] = dt1[col].cat.codes.astype("int16")
        dt1[col] -= dt1[col].min()

# if not is_train:
#     for day in range(tr_last+1, tr_last+ 28 +1):
#         dt[f"d_{day}"] = np.nan

dt = pd.melt(dt1,
              id_vars = catcols,
              value_vars = [col for col in dt1.columns if col.startswith("d_")],
              var_name = "d",
              value_name = "sales")




dt = dt.merge(cal, on= "d", copy = False)
dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

# dt = pd.merge(dt,cal, on= "d",how='left')
# dt = pd.merge(dt,prices, on = ["store_id", "item_id", "wm_yr_wk"], how = 'left')
# dt['sell_price'] = dt['sell_price'].fillna(0)

In [ ]:
temp = dt.groupby('id').size().reset_index().rename(columns={0:'count'})
temp.sort_values('count',ascending=True)


In [ ]:
dt2 = pd.read_csv(r"../input/m5-forecasting-accuracy/sales_train_validation.csv")

In [ ]:
dt2 = pd.melt(dt2,
              id_vars = 'id',
              value_vars = [col for col in dt2.columns if col.startswith("d_")],
              var_name = "d",
              value_name = "sales")


In [ ]:
dt2['sales'] = dt2['sales'].replace(0,np.NaN)

In [ ]:
dt2['d_int']=dt2['d'].str[2:].astype(int)
dt2 = dt2.sort_values(by=['id','d_int'])

In [ ]:
dt2=dt2.dropna()

In [ ]:
dt2=dt2.drop_duplicates(subset='id',keep='first')


In [ ]:
dt['sales_target'] = dt[["id","sales"]].groupby("id")["sales"].shift(-28)

# lags = 15
# for lag in range(1,lags):
#     dt['lag_'+str(lag)] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

exps = 9
for lag in range(1,exps):
    dt['exp_'+str(lag)] = dt[["id","sales"]].groupby("id")["sales"].transform(lambda x : x.ewm(com=exps/10).mean())

win = [7,30,60]
for w in win:
    dt['roll_'+str(w)] = dt[['id','sales']].groupby("id")['sales'].transform(lambda x : x.rolling(w).mean())

In [ ]:
dt['exp1'] = dt[['id','sales']].groupby("id")['sales'].transform(lambda x : x.ewm(com=0.2).mean())

In [ ]:
exps = 9
for lag in range(1,exps):
    dt['exp_'+str(lag)] = dt[["id","sales"]].groupby("id")["sales"].transform(lambda x : x.ewm(com=exps/10).mean())


In [ ]:
dt = pd.merge(dt,dt2[['id','d_int']],on='id',how='left')


In [ ]:
del(dt2)

In [ ]:
dt[dt.id==dt.id[0]][['sales','sales_target']]

In [ ]:

train=dt[~(dt.sales_target.isna())]

In [ ]:
train1=train.dropna()

In [ ]:
train1.info()

In [ ]:
test=dt[dt.sales_target.isna()]

In [ ]:
X_train = train1.drop(columns=['id','d','date','sales_target' ,'wm_yr_wk', 'weekday'],axis=1)
# X_train = X_train.dropna()
y_train = train1['sales_target']

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]

In [ ]:

np.random.seed(555)

fake_valid_inds = np.random.choice(X_train.index.values, 1_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1000,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [ ]:
m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20)

In [ ]:
# q['exponential_ave'] = q.groupby('id')['x'].transform(lambda x: x.ewm(com=0.2).mean())

In [ ]:
X_train.columns

In [ ]:
test['predictions'] = m_lgb.predict(test[X_train.columns])

test_scr = test[['id','d','predictions']]
test_scr = pd.pivot_table(test, values='predictions', index='id', columns='d').add_prefix('').reset_index()
                        

In [ ]:
test_scr.to_csv('output11.csv',index=False)